In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from math import inf

In [17]:
INPUT_FILE = Path.cwd().parent / "inputs" / "astar_in.txt"

### Helper Functions

In [ ]:
def parse_astar_in(path: Path):
    with open(path, 'r') as f:
        lines = f.readlines()
    
    start, goal = False, False
    start_grid = []
    goal_grid = []
    for a in lines:
        if a.strip() == 'start':
            start = True
            continue
        
        if a.strip() == 'goal':
            goal = True
            continue
        
        if start and not goal:
            start_grid.extend(0 if x == "*" else int(x) for x in a.strip().replace(" ", ""))

        if goal:
            if len(start_grid)== 9:
                goal_grid.extend(0 if x == "*" else int(x) for x in a.strip().replace(" ", ""))
            else:
                print("Error: Start grid should have 9 elements before parsing goal grid.")
                break
    
    if start == False or goal == False:
        print("Error: Input file must contain 'start' and 'goal' sections.")
        print("Please check the input file format.")
        return None, None
    
    return tuple(start_grid), tuple(goal_grid)

In [ ]:
def print_puzzle(state: tuple, N: int = 3, blank: int = 0):
    for r in range(N):
        row = state[r*N:(r+1)*N]
        print(" ".join(" " if x == blank else str(x) for x in row))
    print()

def print_path(path, N: int = 3, blank: int = 0):
    for i, state in enumerate(path):
        print(f"Step {i}:")
        print_puzzle(state, N=N, blank=blank)


In [ ]:
def neighbors(state: tuple, N: int):
    blank_tile_index = state.index(0)
    blank_tile_index_r, blank_tile_index_c = divmod(blank_tile_index, N)

    successors = []

    for direction_r, direction_c in [(-1,0),(1,0),(0,-1),(0,1)]:
        new_r, new_c = blank_tile_index_r+direction_r, blank_tile_index_c+direction_c
        if 0 <= new_r < N and 0 <= new_c < N:
            nz = new_r*N + new_c
            lst = list(state)
            lst[blank_tile_index], lst[nz] = lst[nz], lst[blank_tile_index]
            successors.append(tuple(lst))

    return successors

In [ ]:
def reconstruct(parent, goal: tuple):
    path = []
    cur = goal
    while cur is not None:
        path.append(cur)
        cur = parent[cur]
    return path[::-1]

In [ ]:
def astar_algorithm(start: tuple, goal: tuple, heuristic_fn: callable):
    OPEN = []
    CLOSED = set()

    N = int(len(start) ** 0.5) if int(len(start) ** 0.5) == int(len(goal) ** 0.5) else None

    if N is None:
        print("Error: Start and goal states must have the same number of tiles.")
        return None

    g = {start: 0}
    parent = {start: None}

    OPEN.append(start)
    nodes_generated = 1

    # h_cache maps state -> heuristic value
    h_cache = {}

    def h(state):
        if state not in h_cache:
            h_cache[state] = heuristic_fn(state, goal, N)
        return h_cache[state]
    
    while True:
        if not OPEN:
            raise Exception("No solution found.")

        n = min(OPEN, key=lambda s: g[s] + h(s))
        OPEN.remove(n)
        CLOSED.add(n)

        h_val = h(n)
        f_val = g[n] + h_val
        print(f"Expanding: g={g[n]}  h={h_val}  f={f_val}")

        if n == goal:
            print("Nodes generated:", nodes_generated)
            return reconstruct(parent, n), N

        for ni in neighbors(n, N):
            nodes_generated += 1

            tentative_g = g[n] + 1

            # STEP 6/7 combined
            if ni not in OPEN and ni not in CLOSED:
                g[ni] = tentative_g
                parent[ni] = n
                OPEN.append(ni)

            elif tentative_g < g.get(ni, inf):
                g[ni] = tentative_g
                parent[ni] = n

                if ni in CLOSED:
                    CLOSED.remove(ni)
                    OPEN.append(ni)


### Misplaced Tile - Heuristics Function

In [ ]:
def h_misplaced(state, goal, N=None):
    count = 0
    for s, g in zip(state, goal):
        if s != 0 and s != g:
            count += 1
    return count

In [ ]:
start, goal = parse_astar_in(INPUT_FILE)

path, N = astar_algorithm(start, goal, h_misplaced)
print_path(path, N=N)

Expanding: g=0  h=7  f=7
Expanding: g=1  h=7  f=8
Expanding: g=1  h=7  f=8
Expanding: g=1  h=7  f=8
Expanding: g=1  h=7  f=8
Expanding: g=2  h=6  f=8
Expanding: g=2  h=7  f=9
Expanding: g=2  h=7  f=9
Expanding: g=2  h=7  f=9
Expanding: g=2  h=7  f=9
Expanding: g=2  h=7  f=9
Expanding: g=3  h=6  f=9
Expanding: g=3  h=6  f=9
Expanding: g=3  h=6  f=9
Expanding: g=2  h=8  f=10
Expanding: g=2  h=8  f=10
Expanding: g=3  h=7  f=10
Expanding: g=3  h=7  f=10
Expanding: g=3  h=7  f=10
Expanding: g=4  h=6  f=10
Expanding: g=4  h=6  f=10
Expanding: g=4  h=6  f=10
Expanding: g=4  h=6  f=10
Expanding: g=4  h=6  f=10
Expanding: g=4  h=7  f=11
Expanding: g=4  h=7  f=11
Expanding: g=3  h=8  f=11
Expanding: g=3  h=8  f=11
Expanding: g=4  h=7  f=11
Expanding: g=4  h=7  f=11
Expanding: g=4  h=7  f=11
Expanding: g=4  h=7  f=11
Expanding: g=4  h=7  f=11
Expanding: g=5  h=6  f=11
Expanding: g=5  h=6  f=11
Expanding: g=5  h=6  f=11
Expanding: g=5  h=6  f=11
Expanding: g=5  h=6  f=11
Expanding: g=5  h=6  f=11


### Manhattan Distance Heuristic Function

In [9]:
def h_manhattan(state, goal, N):
    distance = 0
    goal_positions = {
        tile: divmod(i, 3)
        for i, tile in enumerate(goal)
    }
    for i, tile in enumerate(state):
        if tile == 0:
            s_r, s_c = divmod(i, N)
            g_r, g_c = goal_positions[tile]

            distance += abs(s_r - g_r) + abs(s_c - g_c)
    return distance

In [ ]:
start, goal = parse_astar_in(INPUT_FILE)

path, N = astar_manhattan(start, goal)
print_path(path, N=N)

Expanding: g=0  h=0  f=0
Expanding: g=1  h=1  f=2
Expanding: g=1  h=1  f=2
Expanding: g=1  h=1  f=2
Expanding: g=1  h=1  f=2
Expanding: g=2  h=2  f=4
Expanding: g=2  h=2  f=4
Expanding: g=2  h=2  f=4
Expanding: g=2  h=2  f=4
Expanding: g=2  h=2  f=4
Expanding: g=2  h=2  f=4
Expanding: g=2  h=2  f=4
Expanding: g=2  h=2  f=4
Expanding: g=3  h=1  f=4
Expanding: g=3  h=1  f=4
Expanding: g=3  h=1  f=4
Expanding: g=3  h=1  f=4
Expanding: g=3  h=1  f=4
Expanding: g=3  h=1  f=4
Expanding: g=3  h=1  f=4
Expanding: g=3  h=1  f=4
Expanding: g=4  h=0  f=4
Expanding: g=4  h=0  f=4
Expanding: g=4  h=0  f=4
Expanding: g=4  h=0  f=4
Expanding: g=4  h=0  f=4
Expanding: g=4  h=0  f=4
Expanding: g=4  h=0  f=4
Expanding: g=4  h=0  f=4
Expanding: g=4  h=2  f=6
Expanding: g=4  h=2  f=6
Expanding: g=4  h=2  f=6
Expanding: g=4  h=2  f=6
Expanding: g=4  h=2  f=6
Expanding: g=4  h=2  f=6
Expanding: g=4  h=2  f=6
Expanding: g=4  h=2  f=6
Expanding: g=5  h=1  f=6
Expanding: g=5  h=1  f=6
Expanding: g=5  h=1  f=6
